In [103]:
pip install gym

  Created wheel for gym: filename=gym-0.17.1-cp37-none-any.whl size=1648719 sha256=9368d89f997a0f20d3fd1d1b5b609905272bd2555eb591f7439078b7e559339f
  Stored in directory: C:\Users\Admin\AppData\Local\pip\Cache\wheels\c0\84\61\523b92d88787ae29689b3cc08cf445d8d8186d7fbe1acbf87b
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [213]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import torch.autograd as autograd
import random

import gym

from collections import deque
from sklearn.preprocessing import StandardScaler

In [177]:
df = pd.read_csv('Sweden Load Data 2005-2017.csv')
df.rename({'cet_cest_timestamp':'time','SE_load_actual_tso':'load'},axis='columns',inplace=True)

df['time'] = pd.to_datetime(df['time'],errors='ignore', utc=True)
df['weekday'] = df['time'].dt.weekday

df.head()

,time,load,weekday
0,2005-01-01 00:00:00+00:00,15991.34,5
1,2005-01-01 01:00:00+00:00,15673.12,5
2,2005-01-01 02:00:00+00:00,15431.81,5
3,2005-01-01 03:00:00+00:00,15326.48,5
4,2005-01-01 04:00:00+00:00,15468.59,5


In [237]:
class DQN(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.fc = nn.Sequential(
            nn.Linear(self.input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, self.output_dim)
        )
        
#         self.fc = nn.Sequential(
#             nn.Linear(self.input_dim, 32),
#             nn.ReLU(),
#             nn.Linear(32, 32),
#             nn.ReLU(),
#             nn.Linear(32, self.output_dim)
#         )

    def forward(self, state):
        qvals = self.fc(state)
        return qvals

In [179]:
class BasicBuffer:

    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)

    def push(self, state, action, reward, next_state, done):
        experience = (state, action, np.array([reward]), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        batch = random.sample(self.buffer, batch_size)

        for experience in batch:
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)

        return (state_batch, action_batch, reward_batch, next_state_batch, done_batch)

    def sample_sequence(self, batch_size):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        min_start = len(self.buffer) - batch_size
        start = np.random.randint(0, min_start)

        for sample in range(start, start + batch_size):
            state, action, reward, next_state, done = self.buffer[start]
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)

        return (state_batch, action_batch, reward_batch, next_state_batch, done_batch)

    def __len__(self):
        return len(self.buffer)

In [201]:
class Env:
    def __init__(self,array_loads,array_weekdays,charge=400,action=None):
        ###################
        self.array_loads = array_loads
        self.array_weekdays=array_weekdays
        self.charge = charge
        self.action = action
        ###################
        self.load_idx = 0
        self.day_idx = 0
        self.collection_actual_load = []
        self.collection_rewards = []
        self.actual_load=None
        self.reward = 0
        self.collection_rewards.append(self.reward)
        self.episode=0
        self.state_size=2
        self.battery_bandwith = 100
        ####################
        self.action_space = np.array([0, 1, 2]) #np.array(['charge','discharge','wait'])
    
    def increase_load_counter(self):
        self.load_idx+=1
        
    def increase_day_counter(self):
        self.day_idx+=1
        
    def generate_input_load(self):
        self.input_load = self.array_loads[self.load_idx]  
            
    def calculate_reward(self):
        if self.charge >= 0 and self.charge <= 400:
            self.reward += np.abs(self.calculate_cost(self.input_load) - self.calculate_cost(self.actual_load))
            self.collection_rewards.append(self.reward)
        else:
            self.reward -= 1000000000
    
    def reset(self):
        self.__init__(self.array_loads, self.array_weekdays)
        self.input_load = self.array_loads[self.load_idx]
        self.actual_load = self.input_load
        initial_state = self.input_load, self.charge
        return initial_state
    
    def step(self, action):
        self.action = action
        self.increase_load_counter()
        self.generate_input_load()
        
        if self.action == 2: #'wait':
            self.actual_load = self.input_load
            self.collection_actual_load.append(self.input_load)
            
        elif self.action == 1: #'discharge':
            self.actual_load = self.input_load - self.battery_bandwith
            self.collection_actual_load.append(self.actual_load)
            self.charge-=100
            
        elif self.action == 0: #'charge':
            self.actual_load = self.input_load + self.battery_bandwith
            self.collection_actual_load.append(self.actual_load)
            self.charge+=100
        
        next_state = self.input_load, self.charge
        
        self.calculate_reward()
        reward = self.reward
        
        done = False
        if self.load_idx >= self.array_loads.shape[0]:
            done = True
            
        return next_state, reward, done
    
    def calculate_cost(self, load):
        return 5 + 0.5*load + 0.05 * load**2

In [240]:
class DQNAgent:
    #gamma=0.99
    def __init__(self, env, use_conv=True, learning_rate=3e-4, gamma=0.99, buffer_size=10000):
        self.env = env
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.replay_buffer = BasicBuffer(max_size=buffer_size)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#         self.use_conv = use_conv
#         if self.use_conv:
#             self.model = ConvDQN(env.observation_space.shape, env.action_space.n).to(self.device)
#         else:
        self.model = DQN(env.state_size, env.action_space.shape[0]).to(self.device)

        self.optimizer = torch.optim.Adam(self.model.parameters())
        self.MSE_loss = nn.MSELoss()

    def get_action(self, state, eps=0.20):
        state = torch.FloatTensor(state).float().unsqueeze(0).to(self.device)
#         print(state)
        qvals = self.model.forward(state)
        action = np.argmax(qvals.cpu().detach().numpy())
        
        if(np.random.randn() < eps):
            return np.random.choice(self.env.action_space)

        return action

    def compute_loss(self, batch):
        states, actions, rewards, next_states, dones = batch
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones)

        curr_Q = self.model.forward(states).gather(1, actions.unsqueeze(1))
        curr_Q = curr_Q.squeeze(1)
        next_Q = self.model.forward(next_states)
        max_next_Q = torch.max(next_Q, 1)[0]
        expected_Q = rewards.squeeze(1) + self.gamma * max_next_Q

        loss = self.MSE_loss(curr_Q, expected_Q)
        return loss

    def update(self, batch_size):
        batch = self.replay_buffer.sample(batch_size)
        loss = self.compute_loss(batch)
#         print(loss)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
scaler = StandardScaler()
scaler.fit([[0, -3*400], [df.load.max(), 3*400]])

env = Env(df.load, df.weekday)
agent = DQNAgent(env)

episodes = 20
batch_size = 32
episode_rewards = []

for e in range(episodes):
    state = env.reset()
    episode_reward = 0
    
    while True:
#         print(state)
        state_norm=scaler.transform([state])
#         print(tuple(state_norm[0]))
        action = agent.get_action(tuple(state_norm[0]))
#         print(action)
        next_state, reward, done = env.step(action)
        next_state_norm = scaler.transform([next_state])
        agent.replay_buffer.push(tuple(state_norm[0]), action, reward, tuple(next_state_norm[0]), done)
        episode_reward += reward

        if len(agent.replay_buffer) > batch_size:
            agent.update(batch_size) 

        if done:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break

        state = next_state
              

In [120]:
env_id = "CartPole-v0"
env_test = gym.make(env_id)

env_test.reset()

array([ 0.03983449, -0.01862387,  0.0072048 ,  0.04002884])

In [173]:
load = 900

c1 = 5 + 0.5*load + 0.05 * load**2
c1 

40955.0

In [172]:
load = 1000

c2 = 5 + 0.5*load + 0.05 * load**2
c2

50505.0

In [174]:
c1 - c2

-9550.0